In [1]:
from pathlib import Path

import re
from pprint import pprint

import sys
src_path = str(Path.cwd().parent / "src")
sys.path.append(src_path)

import csv 

In [2]:
from pdf_processing import *

TODO: 
 - check more PDFs for differences in format
 - write documentation
 - locations not included after the date (might need to be taken care of manually)
 - maybe use translate() instead of pdf.replace()
 - footnotes (like in People_of_Greece.pdf and Senate_Floor_Immigration_Reform.pdf)



DONE: 
 - Q&As / interviews
     - function to count the number of occurrences of "Obama:", "Question:", etc.
   
      
 - locations with multiple comma's (ex. when Washington, D.C. is included) --> might be better to store location as one elements instead of splitting it up, the location can be cleaned up afterwards
     - location is not split up anymore, but kept as a single element (will be easier to clean up afterwards)

In [3]:
# pdfs that have these in the title might have to be excluded: interview, presser, Press_Conference, press, meeting, 
# debate, hall (maybe)
multiple_people_talking = ['Barack_Obama_-_Al-Arabiya_Interview.pdf', 'CGI_2013.pdf', 
                           'Midterm_Elections_Presser_2014.pdf', 'YSEALI_Fellows.pdf', 
                           'PM_Abe_of_Japan_Joint_Presser.pdf', 'Paris_Press_Conference_2015.pdf',
                           'Gun_Violence_Presser.pdf', 'Associated_Press_Luncheon.pdf', 'Final_Press_Conference.pdf',
                           'CNN_Guns_in_America_Town_Hall.pdf', 'START_Treaty_Presser.pdf', 
                           'Midterm_Elections_Presser_2010.pdf', 'Joint_Presser_with_President_Benigno_Aquino.pdf',
                           'Worldwide_Troop_Talk.pdf', 'Post_ASEAN_Presser_2016.pdf', 'Post_G7_Presser_Japan.pdf',
                           'Presser_Unannounced_Heath_Care.pdf', 'Kenya_Civil_Society_Meeting.pdf',
                           'Obama-Romney_-_Second_Live_Debate.pdf', 'Jamaica_Town_Hall.pdf', 'YSEALI_Town_Hall.pdf',
                           'BP_Oil_Spill_Presser.pdf', 'VA_Misconduct_Allegations.pdf', 'ISIL_Pentagon_Update.pdf',
                           'ISIL_Presser_Post_Security_Council_Meeting.pdf']
# maybe: 'Alabama_Tornado_Devestation.pdf', 'NAACP_Coference_2015.pdf'

# the year for Community_College_Plan.pdf is 20105 but should be 2015

Get the path to the directory in which the PDFs are stored.

In [4]:
pdf_dir = Path.cwd().parent / "pdfs"

Filepaths of all the PDFs in the folder `pdf_dir`, can be used to iterate over all the PDFs to store the extracted speeches in a dataframe.

In [5]:
pdfs = list(pdf_dir.glob('*.pdf'))  
print("current number of PDFs:", len(pdfs))

current number of PDFs: 436


Get filepath of the PDF you want to process.

In [6]:
filepath = pdfs[116]
print("File:", str(filepath.parts[-1]))

File: Auschwitz_and_Birkenau_Liberation_65th.pdf


Make a PDFHandler object for the given filepath.

In [7]:
pdf = PDFHandler(filepath)

Print the last page of the PDF before it has been processed.

In [8]:
print(pdf.original_page(-1))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
For you know the truth that Elie Wiesel spoke when I stood with him at Buchenwald last 
spring.  There, where his father and so many innocent souls left this earth, Elie said: 
 
"...memory has become [a] sacred duty of all people of goodwill.”
We have a sacred duty to remember the twisted thinking that led here -- how a great society 
of culture and science succumbed to the worst instincts of man and rationalized mass murder 
and one of the most barbaric acts in history. 
We have a sacred duty to remember the cruelty that occurred here, as told in the simple 
objects that speak to us even now: the suitcases that still bear their names; the wooden clogs 
they wore; the round bowls from which they ate; those brick buildings from which there was 
no escape -- where so many Jews died with Sh’ma Israel on their lips; and the very earth at 
Auschwitz, which is still hallowed by their ashes -- Jews and those who tried to save them, 
Polish a

Print the first page of the PDF before it has been processed.

In [9]:
print(pdf.original_page(0))

  
AA RR
mmeerriiccaann hheettoorriicc..ccoomm  
 
Barack Obama 
Honoring 65th Anniversary of Auschwitz and Birkenau Liberation 
 
delivered 27 January 2010
 
 
AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio 
Good morning. And thank you to everyone who worked to bring us to this day, especially the 
 
International Auschwitz Council and the Auschwitz-Birkenau State Museum.
To President Kacynski, Prime Minister Tusk, and to the people of Poland: Thank you for 
preserving a place of such great pain for the Polish people, but a place of remembrance and 
 
learning for the world.
Although I can’t be with you in person, I am proud that the United States is represented there 
today by a delegation of distinguished Americans, including Ambassador Feinstein; my wife 
Michelle’s chief of staff, Susan Sher; and my good friend, and the son of Holocaust survivors, 
 
Julius Genachowski.
And let me commend you for recognizing a woman who has devoted her life to preservin

Define a regular expression to get the date, location, and content of the speech. Extract the entire speech from the PDF.

In [10]:
start = r"^(?:\s*AA *RR\s+mmeerriiccaann *hheettoorriicc\.\.ccoomm|\s*AmericanRhetoric\.com|\s*AAmmeerriiccaannRRhheettoorriicc\.\.ccoomm)?"

datemarker =r"^(delivered|First *Broadcast|Uploaded[ a-z]*)?"
date = r"(?P<day>[0-9]{1,2}) *(?P<mon>[a-z]+) *(?P<year>[0-9]{2,})"
loc = r"(, *(?P<location>.+?))?"
auth = r"(?:.?AUTHENTICITY CERTIFIED: Text version below transcribed directly from audio( *and edited for continuity)?.?)?"

content = r"(?P<content>.*)"

#footnotes = r"^(([1-9][0-9]*.+?)+\n+)?"
tr = r"(Transcription *by *[a-z.\- ]+\s*)?"
url = r"((Property *of *)?AmericanRhetoric\.com)?"
page = r"(page *[0-9]+\s*$)"

full_pat = start + \
r"(\s+.*?" + datemarker + " *" + date + loc + r"\s+" + auth + r")?" + \
r"\s+" + content + \
r"^(?:\s*" + tr + url + r".*?" + page + r")"
pat = re.compile(full_pat, re.I | re.DOTALL | re.M)

speech = pdf.extract_speech(pat)
print(speech)

Good morning. And thank you to everyone who worked to bring us to this day, especially the 
 
International Auschwitz Council and the Auschwitz-Birkenau State Museum.
To President Kacynski, Prime Minister Tusk, and to the people of Poland: Thank you for 
preserving a place of such great pain for the Polish people, but a place of remembrance and 
 
learning for the world.
Although I cant be with you in person, I am proud that the United States is represented there 
today by a delegation of distinguished Americans, including Ambassador Feinstein; my wife 
Michelles chief of staff, Susan Sher; and my good friend, and the son of Holocaust survivors, 
 
Julius Genachowski.
And let me commend you for recognizing a woman who has devoted her life to preserving the 
lessons of the shoah for future generations -- Sara Bloomfield of the United States Holocaust 
 
Memorial Museum in Washington.
But most of all, I want to thank those of you who found the strength to come back again, so 
many years 

Print the relevant info of the PDF.

In [11]:
pdf.print_info()

Title: Auschwitz_and_Birkenau_Liberation_65th
Number of pages: 2
Date: 27 January 2010
Location: unknown_location


Check whether there are multiple speakers. (If there is a high "Obama:" count, this might indicate he is answering questions, i.e., it is an interview, press conference, debate,...)

In [12]:
count = ('obama:', 'president:', 'question:', 'audience:', 'member:')
counts, speakers = pdf.multiple_speakers(speech, count)
print(counts)
print()
pprint(speakers)

{'obama:': 0, 'president:': 0, 'question:': 0, 'audience:': 0, 'member:': 0}

['poland:', 'said:', 'now:', 'good:', 'you:', 'memorials:']


Replace or delete some characters to clean the speech.

In [13]:
old = [r'-+', r'\.{2,}', r'[’‘]', r'"', r'’’', r'‘‘', r'“', r'”', r'\[sic\]', r'\s+', r'\[Source:\s*http.*\]', r'[\[\]\(\)]']
new = [r' ' , r' '     , r"'"   , r'' , r''  , r''  , r'' , r'' , r' '      , r' '  , r'',                     r''      ]

clean_speech = pdf.replace(speech, old, new)
print(clean_speech)

Good morning. And thank you to everyone who worked to bring us to this day, especially the International Auschwitz Council and the Auschwitz Birkenau State Museum. To President Kacynski, Prime Minister Tusk, and to the people of Poland: Thank you for preserving a place of such great pain for the Polish people, but a place of remembrance and learning for the world. Although I cant be with you in person, I am proud that the United States is represented there today by a delegation of distinguished Americans, including Ambassador Feinstein; my wife Michelles chief of staff, Susan Sher; and my good friend, and the son of Holocaust survivors, Julius Genachowski. And let me commend you for recognizing a woman who has devoted her life to preserving the lessons of the shoah for future generations Sara Bloomfield of the United States Holocaust Memorial Museum in Washington. But most of all, I want to thank those of you who found the strength to come back again, so many years later, despite the h

In [14]:
csv_filename = Path.cwd().parent / "speeches_csv" / "speeches.csv"

fields = ["title", "pages", "date", "location", "highest_speaker_count", "content"]

# use 'a+' instead of 'w' if you want to append at the bottom of the file instead of replacing the file
with open(csv_filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL) 
        
    csvwriter.writerow(fields) 
        
    for i in range(10):  # for i in range(len(pdfs)):
        filepath = pdfs[i]
        pdf = PDFHandler(filepath)
        speech = pdf.extract_speech(pat)
        
        count = ('obama:', 'president:', 'question:', 'audience:', 'member:')
        counts, speakers = pdf.multiple_speakers(speech, count)
        
        clean_speech = pdf.replace(speech, old, new)
        
        title = pdf.get_title()
        nb_pages = pdf.get_nb_pages()
        date = pdf.get_date_str()
        loc = pdf.get_location_str()
        highest_count = max(counts.values())
        
        row = [title, nb_pages, date, loc, highest_count, clean_speech]
        csvwriter.writerow(row)
        
        print("pdf", i+1)
        print(row[:-1])
        print(row[-1][:100])
        print()

pdf 1
['CGI_2013', 19, '24 September 2013', 'Sheraton New York Hotel and Towers, New York, New York', 10]
Hillary Clinton: Thank you very much. I have the great pleasure to introduce our next two speakers, 

pdf 2
['Prayer_Breakfast_2016', 7, '4 February 2016', 'Washington Hilton, Washington, D.C.', 0]
Well, good morning. Giving all praise and honor to God for bringing us together here this morning. I

pdf 3
['Security_Team_Announcement', 5, '1 December 2008', 'Chicago, Illinois', 0]
Good morning, everybody. I hope you all had a wonderful Thanksgiving. Last week, we announced our ec

pdf 4
['Cairo_University', 14, '4 June 2009', 'Cairo, Egypt', 0]
Thank you so much. Good afternoon. I am honored to be in the timeless city of Cairo, and to be hoste

pdf 5
['Umpqua_Community_College_Shootings', 4, '1 October 2015', 'Washington, D.C.', 0]
Theres been another mass shooting in America this time, in a community college in Oregon. That means

pdf 6
['White_House_Correspondent_Dinner_2013', 6, 